In [7]:
from tensorflow.keras import datasets
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras import models, metrics, losses
import numpy as np
import tensorflow as tf


(x_train, y_train),(x_test, y_test) = datasets.fashion_mnist.load_data()

In [9]:
def preprocess(imgs):
    imgs = imgs.astype("float32") / 255.0
    imgs = np.pad(imgs,((0, 0), (2, 2), (2, 2)), constant_values=0.0)
    imgs = np.expand_dims(imgs, -1)
    return imgs
x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [10]:
# In this snippet, K.random_normal is a Keras function to generate random values following
# a Gaussian distribution. 'shape' indicates the shape of the output tensor. It defines how many
# random values we want to generate
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [11]:
# Defining the encoder
encoder_input = layers.Input(
    shape=(32, 32, 1), name="encoder_input"
)
x = layers.Conv2D(32,(3,3),strides=2,activation="relu",padding="same")(encoder_input)
x = layers.Conv2D(64, (3,3),strides=2,activation="relu",padding="same")(x)
x = layers.Conv2D(128,(3,3),strides=2,activation="relu",padding="same")(x)

shape_before_flattening = K.int_shape(x)[1:]
x = layers.Flatten()(x)
z_mean = layers.Dense(2,name="z_mean")(x)
z_log_var = layers.Dense(2,name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")

In [12]:
# Here we want to define the decoder
decoder_input = layers.Input(shape=(2,), name="decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    128, (3,3),strides=2,activation="relu",padding="same"
)(x)
x = layers.Conv2DTranspose(
    64, (3,3),strides=2,activation="relu",padding="same"
)(x)
x = layers.Conv2DTranspose(
    32, (3,3),strides=2,activation="relu",padding="same"
)(x)
decoder_output = layers.Conv2D(
    1,
    (3,3),
    strides=1,
    activation="sigmoid",
    padding="same",
    name="decoder_output",
)(x)
decoder = models.Model(decoder_input, decoder_output)
decoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 2048)              6144      
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose_3 (Conv2DT  (None, 8, 8, 128)        147584    
 ranspose)                                                       
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 16, 16, 64)       73792     
 ranspose)                                                       
                                                                 
 conv2d_transpose_5 (Conv2DT  (None, 32, 32, 32)       1846

In [13]:
# Building the VAE as a subclass of the Keras Model class
class VAE(models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")
        
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker
        ]
    
    def call(self, inputs):
        z_mean, z_log_var, z = encoder(inputs)
        reconstruction = decoder(z)
        return z_mean, z_log_var, reconstruction
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, reconstruction = self(data)
            reconstruction_loss = tf.reduce_mean(500 * losses.binary_crossentropy(data, reconstruction, axis=(1,2,3)))
            kl_loss = tf.reduce_mean(
                tf.reduce_sum(
                    -0.5
                    * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                    axis = 1,
                )
            )
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {m.name: m.result() for m in self.metrics}

In [20]:
# Benchmark to beat: total_loss: 131.5970 - reconstruction_loss: 126.3598 - kl_loss: 5.2372, achieved with Adam
# Using adam optimizer, with epochs 7, batch_size 200 we got total_loss: 129.8120 - reconstruction_loss: 124.4130 - kl_loss: 5.399
from tensorflow.keras import optimizers

vae = VAE(encoder, decoder)
vae.compile(optimizer="adam")
vae.fit(
    x_train,
    epochs=7,
    batch_size=200,
    shuffle=True,
    validation_data=(x_test, x_test),
)

Epoch 1/7
300/300 [==============================] - 75s 248ms/step - total_loss: 129.6915 - reconstruction_loss: 124.2881 - kl_loss: 5.4035 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 2/7
300/300 [==============================] - 79s 264ms/step - total_loss: 129.5183 - reconstruction_loss: 124.0685 - kl_loss: 5.4499 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 3/7
300/300 [==============================] - 72s 239ms/step - total_loss: 129.3864 - reconstruction_loss: 123.9423 - kl_loss: 5.4441 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 4/7
300/300 [==============================] - 73s 243ms/step - total_loss: 129.2545 - reconstruction_loss: 123.7941 - kl_loss: 5.4603 - val_total_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_kl_loss: 0.0000e+00
Epoch 5/7
300/300 [==============================] - 77s 256ms/s